In [3]:
import os 
import re
import time
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import ik_parsing as ik
head, tail = os.path.split(os.getcwd())
list_of_filenames = os.listdir(os.path.join(head, '2019', '2019'))

C:\Users\Nick\Anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [4]:
import importlib
importlib.reload(ik)

<module 'ik_parsing' from 'C:\\Users\\Nick\\Desktop\\Chicago\\DIME\\kanoon\\ik_parsing.py'>

In [5]:
# We loop over each file and find all citations using regex and add it to the citations dictionary with the case 
# as the key and a set of citations as the value.

network = {}
titles = []
dates = []
courts = []
cnr_nums = []
case_nums = []
petitioner = []
respondent = []
petitioner_advocates = []
respondent_advocates = []
judges = []
count = 0

start_time = time.time()  

for filename in list_of_filenames:
    count+=1
    if count>500:
        break
    with open(os.path.join(os.path.join(head, '2019', '2019', filename)), encoding="utf8") as f:
        soup = BeautifulSoup(f, 'html.parser')
        text = soup.get_text().lower()
#         if count==2:
#             print(text)
#             break
        try:
            courts.append(soup.find("div", {"class": "docsource_main"}).string)
            title, date = soup.title.string.split(' on ')
            titles.append(title)
            dates.append(date)
        except AttributeError:
            titles.append(np.nan)
            dates.append(np.nan)
            courts.append(np.nan)
#         print(soup.get_text())
#         print(soup.title.string)
#         print(soup.find("div", {"class": "docsource_main"}).string)
        case_nums.append(ik.extract_case_num(text))
        petitioner.append(ik.extract_petitioner(text))
        respondent.append(ik.extract_respondent(text))
        pet_adv, resp_adv = ik.extract_petitioner_advocate(text)
        petitioner_advocates.append(pet_adv)
        respondent_advocates.append(resp_adv)
        cnr_nums.append(ik.extract_cnr(text))
        judges.append(ik.extract_judges(text))
             
    
print(f"--- {(time.time() - start_time)} seconds ---")  # takes 1000-1500 seconds

# With 5000 files it takes 60-220 seconds to append to lists. 

--- 9.89113998413086 seconds ---


In [6]:
df = pd.DataFrame({'file':list_of_filenames[:500], 'cnr_num': cnr_nums, 'title': titles, 'date': dates, 'court': courts, 
                   'case_number': case_nums, 'petitioner': petitioner, 'respondent':respondent, 
                   'petitioner_advocate': petitioner_advocates, 'respondent_advocate': respondent_advocates,
                   'judge': judges})
df['bench_size'] = df.loc[:,'judge'].dropna().apply(len)
df['chief_justice'] = df.loc[:,'judge'].dropna().apply(lambda x: (any('chief' in item for item in x)))

keywords_state = ['state', 'government', 'commissioner', 'national', 'india', 'indian', 'public', 'magistrate']
df['state'] = df.loc[:,'title'].dropna().apply(lambda x: (any(keyword in x.lower() for keyword in keywords_state)))

keywords_business = ['pvt', 'private', 'limited', 'ltd', 'company', 'co.', 'llc']
df['petitioner_business'] = df.loc[:,'petitioner'].dropna().apply(lambda x: (any(keyword in x.lower() for keyword in keywords_business)))
df['respondent_business'] = df.loc[:,'respondent'].dropna().apply(lambda x: (any(keyword in x.lower() for keyword in keywords_business)))
df['petitioner'] = ik.preprocess_company_names(df.loc[:, 'petitioner'].tolist())
df['respondent'] = ik.preprocess_company_names(df.loc[:, 'respondent'].tolist())

keywords_male = ['shri', 'mr.', 'master']
keywords_female = ['smt', 'mrs.', 'ms']

df.replace('\n', ' ', regex = True, inplace = True)
df.head() 

,file,cnr_num,title,date,court,case_number,petitioner,respondent,petitioner_advocate,respondent_advocate,judge,bench_size,chief_justice,state,petitioner_business,respondent_business
0,100006976.txt,gahc010142142019,The Commissioner Of Central Goods ... vs Dhara...,"25 October, 2019",Gauhati High Court,case no. : i.a.(civil) 2085/2019,1 the commissioner of central goods and servic...,1 dharampal satyapal s no 6 7 ...,mr. s c keyal,mr. p baruah -,"{ mr. justice achintya malla bujor barua, the...",2.0,True,True,False,True
1,100009202.txt,gahc010130082013,Jufiul Huda vs The State Of Assam And 3 Ors,"11 March, 2019",Gauhati High Court,case no. : wp(c) 2866/2013,1 jufiul huda s/o nurul huda r/o s...,1 the state of assam and 3 ors rep...,ms.s sultana,mr.s b sarma ...,{ mr. justice arup kumar goswami},1.0,False,True,False,False
2,100013560.txt,gahc010076622019,Amena Khatun vs Union Of India And 5 Ors,"29 April, 2019",Gauhati High Court,case no. : wp(c) 2410/2019,1 amena khatun w/o hamed ali @ ab...,1 union of india and 5 ors rep b...,mr h r a choudhury ...,asstt.s.g.i. ...,"{ mr. justice manish choudhury, mr. justice m...",2.0,False,True,False,False
3,100015777.txt,gahc010021642017,Raj Kumar Singh vs The State Of Assam And Anr,"3 June, 2019",Gauhati High Court,case no. : crl.rev.p. 140/2017,1 raj kumar singh s/o sri hetrm si...,1 the state of assam and anr 2 ...,ms.r devi ...,"pp, assam ...",{ mrs. justice rumi kumari phukan},1.0,False,True,False,False
4,100020544.txt,gahc010133782017,National Insurance Co. Ltd vs Chittarlekha Bor...,"30 April, 2019",Gauhati High Court,case no. : i.a.(civil) 2261/2017,1 national insurance co regd o...,1 chittarlekha borah and ors w/o ...,mr.s s sarma,mrs.g sarma,{ mr. justice nani tagia},1.0,False,True,True,False


In [7]:
import csv
import textdistance

start_time = time.time()  

assam_companies = pd.read_csv('../company/Copy of Assam_2016.csv')
assam_companies_names = ik.preprocess_company_names(assam_companies.COMPANY_NAME.tolist())

# print(assam_companies_names[:5])

list_of_litigants = df[df.petitioner_business==True].loc[:,'petitioner'].tolist()  
list_of_litigants.extend(df[df.respondent_business==True].loc[:,'respondent'].tolist())
print(len(list_of_litigants))

list_of_litigants = ik.preprocess_company_names(list_of_litigants)

tf = {}
for litigant in list_of_litigants:
    for word in litigant.split():
        if word in tf:
            tf[word] += 1
        else:
            tf[word] = 1
        
# temp_list = ['AB ( INDIA) MULTITRADE)', 'THREE LEAVES INDIA PVT LTD']
temp_list = ['three leaves india pvt ltd']
    

df['max_score_petitioner'], df['business_match_petitioner'] = (np.nan, np.nan)
df.business_match_petitioner = df[df.petitioner_business==True].loc[494:496,'petitioner'].apply(ik.match_business_tf, args=(assam_companies_names[:],tf))

#df[df.petitioner_business==True].loc[:,'max_score_petitioner'], df[df.petitioner_business==True].loc[:,'business_match_petitioner'] = 
# df.business_match_petitioner = df[df.petitioner_business==True].loc[:,'petitioner'].apply(ik.match_business, args=(assam_companies_names[:5],))
# df[df.petitioner_business==True].loc[:,'petitioner']


print(f"--- {(time.time() - start_time)} seconds ---")  # 50 companies list and 500 df takes 40s; 500, 500 takes 512s


91
--- 41.99468779563904 seconds ---


In [8]:
df[df.petitioner_business==True]

,file,cnr_num,title,date,court,case_number,petitioner,respondent,petitioner_advocate,respondent_advocate,judge,bench_size,chief_justice,state,petitioner_business,respondent_business,max_score_petitioner,business_match_petitioner
4,100020544.txt,gahc010133782017,National Insurance Co. Ltd vs Chittarlekha Bor...,"30 April, 2019",Gauhati High Court,case no. : i.a.(civil) 2261/2017,1 national insurance co regd o...,1 chittarlekha borah and ors w/o ...,mr.s s sarma,mrs.g sarma,{ mr. justice nani tagia},1.0,False,True,True,False,NaN,NaN
32,100113934.txt,gahc010025052016,M/S Glove Infracon Private Ltd. ... vs The Nat...,"30 January, 2019",Gauhati High Court,case no. : arb.p. 24/2016,1 m/s glove infracon and 2 ors a ...,1 the national small industries corporation ...,mr.s sarma,mrv k barooahr-5 ...,NaN,NaN,NaN,True,True,True,NaN,NaN
33,100115432.txt,gahc010015502009,Oriental Insurance Co. Ltd vs Md. Mehebub Alom...,"21 May, 2019",Gauhati High Court,case no. : mfa 38/2009,1 oriental insurance co a comp...,1 md mehebub alom and anr s/o n...,s k goswami,"md.s islam , mr amarenra gogoi legal aid coun...",{ mr. justice suman shyam},1.0,False,False,True,True,NaN,NaN
36,100117890.txt,gahc010175422015,M/S Abci Infrastructures Pvt Ltd vs The State ...,"4 February, 2019",Gauhati High Court,case no. : wp(c) 1357/2015,1 m/s abci infrastructures form...,1 the state of assam and 2 ors rep...,mr.d senapati,...,{ mr. justice kalyan rai surana},1.0,False,True,True,False,NaN,NaN
38,10012601.txt,gahc010005882018,United India Insurance Co. Ltd vs Mukshed Ali,"24 April, 2019",Gauhati High Court,case no. : i.a.(civil) 50/2018,1 united india insurance co regd an...,1 mukshed ali s/o late manikuddin v...,mr. a j saikia,mr. a r agarwala,{ mr. justice nani tagia},1.0,False,True,True,False,NaN,NaN
72,100230509.txt,gahc010003792013,National Insurance Co. Ltd vs Debashis Das And...,"11 February, 2019",Gauhati High Court,case no. : mfa 17/2013,1 national insurance co a comp...,1 debashis das and anr s/o late ...,mr.b c das,,{ mr. justice suman shyam},1.0,False,True,True,False,NaN,NaN
74,100236009.txt,gahc010120722018,Sbi General Insurance Company Ltd vs Mustt Sar...,"16 September, 2019",Gauhati High Court,case no. : i.a.(civil) 2033/2018,1 sbi general insurance company junc...,1 mustt sarifa begum and 7 ors w/o la...,mr. r goswami,mr. a r agarwala in macapp. 530...,{ mr. justice michael zothankhuma},1.0,False,False,True,False,NaN,NaN
104,100323627.txt,gahc010013892016,Oriental Insurance Co. Ltd vs Kalpana Das And ...,"15 March, 2019",Gauhati High Court,case no. : macapp. 106/2016,1 oriental insurance co having ...,1 kalpana das and 4 ors w/o sri p...,ms.m choudhury,...,{ mr. justice manish choudhury},1.0,False,False,True,False,NaN,NaN
107,100344446.txt,gahc010278082018,Cholamandalam M S General ... vs Smti Kunja Ba...,"29 March, 2019",Gauhati High Court,case no. : macapp. 109/2019,1 cholamandalam m s general insurance co ...,1 smti kunja basumatary and 5 ors mot...,mr. k k bhatta,,{ mr. justice ajit borthakur},1.0,False,False,True,False,NaN,NaN
116,100372117.txt,gahc010033952016,Abdul Latif And 5 Ors vs The State Of Assam An...,"1 April, 2019",Gauhati High Court,case no. : wp(c) 7835/2016,1 abdul latif and 5 ors s/o abdul b...,1 the state of assam and 16 ors rep by the c...,mr.s haque,"sc, co-op. ...",{ mr. justice sanjay kumar medhi},1.0,False,True,True,True,NaN,NaN


In [9]:
tf

{'1': 99,
 'national': 20,
 'insurance': 43,
 'co': 71,
 'regd': 11,
 'office': 117,
 'at': 110,
 'middleton': 10,
 'street': 14,
 'kolkata': 16,
 'and': 169,
 'one': 22,
 'of': 151,
 'the': 204,
 'regional': 47,
 'offices': 3,
 'known': 3,
 'as': 19,
 'guwahati': 98,
 'g': 29,
 's': 88,
 'road': 89,
 'bhangagarh': 15,
 'ghy': 21,
 '5': 30,
 'a': 54,
 'branch': 18,
 'tezpur': 10,
 'assam': 181,
 'm/s': 33,
 'glove': 1,
 'infracon': 1,
 '2': 61,
 'ors': 31,
 'company': 45,
 'incorporated': 8,
 'under': 19,
 'companies': 19,
 'act': 23,
 '1956': 17,
 'having': 69,
 'its': 133,
 'registered': 50,
 '22': 5,
 'prince': 1,
 'commercial': 3,
 'complex': 4,
 'ulubari': 12,
 '781007': 5,
 'dist': 96,
 'kamrupm': 1,
 'ali': 26,
 'ahad': 2,
 'bye': 9,
 'lane': 8,
 'paltan': 11,
 'bazar': 14,
 'dibrugarh': 14,
 '786005': 3,
 'rep': 57,
 'by': 82,
 'managing': 16,
 'director': 25,
 'mr': 6,
 'ahmed': 7,
 'ansari': 4,
 's/o': 52,
 'lt': 17,
 'md': 10,
 'permanent': 3,
 'resident': 4,
 'p': 122,
 'o'